In [1]:
import itertools
import os
import random
import re
import subprocess
from typing import List, Tuple

import IPython.display
from matplotlib.axes import Axes
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from matplotlib.patches import Rectangle

pd.options.display.max_columns = 100
pd.options.display.max_colwidth = 200

# Reading

In [2]:
RUNDIRS = '../logs/rundirs'
  
# RUNNAME = '20241203_133243' 

# RUNNAME = '20241203_170129_all600'
# RUNNAME = '20241213_104400_racing'
# RUNNAME = '20241214_122216_racing_passhum'
# RUNNAME = '20241221_161344_rerouting_racing_passhum'
# RUNNAME = '20241222_105140_rerouting_racing_passhum'
# RUNNAME = '20241226_130310'
RUNNAME = '20241226_161338'

I_MAP = 1

In [3]:
if RUNNAME == '20241203_170129_all600':
    RUNNAME_LINEARIZATIONS = '20241212_114658_lin_abcd'
else:
    RUNNAME_LINEARIZATIONS = None

DIRECTORY_DATA = f'data/{RUNNAME}'
os.makedirs(DIRECTORY_DATA, exist_ok=True)

def read_df_runname(runname):
    rundir = f'{RUNDIRS}/{runname}'
    filename = f'{rundir}/sorted.csv'
    if not os.path.exists(filename):
        subprocess.run(['./consolidate.py', rundir], check=True)
    df_orig = pd.read_csv(filename)
    return df_orig

df_orig_with_bad = read_df_runname(RUNNAME)
df_orig_with_bad

,Date,Scenario ID,Vehicle ID,Vehicle type,Cycle distance (m),No. of completed missions,Total distance traveled (m),No. of stops,No. of forcing events,No. of violations,No. of critical sections,No. of near-misses,No. of collisions,Is blocked,Total waiting time (s),Maximum waiting time (s),Total time (s),Maximum acceleration (m/s^2),Maximum speed (m/s),Average speed (m/s),Scenario,Current datetime,Time passed (real),Time passed (sim.),isCanPassFirstActive,isRacingThroughCrossroadAllowed,probabilitySlowingDownForHuman,Human V0,Collision events,Vehicle size (m),v_current,v_max,can pass first,violation of priorities,moving slowly,improper parking,cautious mode,rerouting at parked / slow,moving backwards,change of priorities,stops,violations,near misses,collisions,"traveled total, m",no. missions,blocked,"position (x, y), m","traveled, m",path index,no. poses,CP (index),"posTo Slow, m","distance ToCP, m",status,future missions,Linearization A,Linearization B,Linearization C,Linearization D1,Linearization D2,Linearization D3,Linearization D0
0,20241226_151426,"map-generator/generated-maps/2024-11-28_13:17:39_with_bridges/scenario7-1.json, baseline, seed 1, probabilityForcingForHuman 0; AUTOMATED_FIRST",0,HumanDrivenVehicle,454.75,5,2384.0,12,0,0,69,0,3,0,605.6,259.2,1598.6,0.3,2.6,1.5,"map-generator/generated-maps/2024-11-28_13:17:39_with_bridges/scenario7-1.json, baseline, seed 1, probabilityForcingForHuman 0; AUTOMATED_FIRST",2024-12-26 15:23:51,0:09:24,0:26:38 (x2.8),"hum=true, aut=false",True,0.0,0 forcing events,"0 minor, 4 major (0:08:51: V0[114]-V3[737], 0:10:18: V2[504]-V3[843], 0:14:19: V0[25]-V2[500], 0:26:21: V0[67]-V3[786])","[length=10.0, width=10.0, safe distance: front=5.0, back=5.0, left=1.0, right=1.0]",2.6,2.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0,3,2383.8,5,NaN,"(466.3, 139.2)",112.6,111,454,NaN,447.0,NaN,DRIVING,"2: [455, 454]",NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,20241226_151426,"map-generator/generated-maps/2024-11-28_13:17:39_with_bridges/scenario7-1.json, baseline, seed 1, probabilityForcingForHuman 0; AUTOMATED_FIRST",1,AutonomousVehicle,436.61,8,7768.3,9,0,0,39,0,0,0,15.4,10.7,1598.6,0.3,5.6,4.9,"map-generator/generated-maps/2024-11-28_13:17:39_with_bridges/scenario7-1.json, baseline, seed 1, probabilityForcingForHuman 0; AUTOMATED_FIRST",2024-12-26 15:23:51,0:09:24,0:26:38 (x2.8),"hum=true, aut=false",True,0.0,0 forcing events,"0 minor, 4 major (0:08:51: V0[114]-V3[737], 0:10:18: V2[504]-V3[843], 0:14:19: V0[25]-V2[500], 0:26:21: V0[67]-V3[786])","[length=10.0, width=10.0, safe distance: front=5.0, back=5.0, left=1.0, right=1.0]",5.6,5.6,NaN,NaN,NaN,NaN,NaN,0 / 0,NaN,NaN,NaN,NaN,0,0,7767.8,8,NaN,"(353.0, 484.6)",796.4,793,871,NaN,840.7,NaN,DRIVING,1: [871],NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,20241226_151426,"map-generator/generated-maps/2024-11-28_13:17:39_with_bridges/scenario7-1.json, baseline, seed 1, probabilityForcingForHuman 0; AUTOMATED_FIRST",2,AutonomousVehicle,1354.80,6,6969.9,11,0,0,59,0,2,0,149.5,80.1,1598.6,0.3,5.6,4.4,"map-generator/generated-maps/2024-11-28_13:17:39_with_bridges/scenario7-1.json, baseline, seed 1, probabilityForcingForHuman 0; AUTOMATED_FIRST",2024-12-26 15:23:51,0:09:24,0:26:38 (x2.8),"hum=true, aut=false",True,0.0,0 forcing events,"0 minor, 4 major (0:08:51: V0[114]-V3[737], 0:10:18: V2[504]-V3[843], 0:14:19: V0[25]-V2[500], 0:26:21: V0[67]-V3[786])","[length=10.0, width=10.0, safe distance: front=5.0, back=5.0, left=1.0, right=1.0]",5.6,5.6,NaN,NaN,NaN,NaN,NaN,4 / 0,NaN,NaN,NaN,NaN,0,2,6969.3,6,NaN,"(389.5, 367.8)",733.1,733,1242,841,809.4,109.3,DRIVING,1: [1135],NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,20241226_151426,"map-generator/generated-maps/2024-11-28_13:17:39_with_bridges/scenario7-1.json, baseline, seed 1, probabilityForcingForHuman 0; AUTOMATED_FIRST",3,AutonomousVehicle,2302.95,4,6764.8,11,0,0,57,0,3,0,224.2,143.3,1598.6,0.3,5.6,4.2,"map-generator/generated-maps/2024-11-28_13:17:39_with_bridges/scenario7-1.json, baseline, seed 1, probabilityForcingForHuman 0; AUTOMATED_FIRST",2024

In [4]:
if 'Linearization C' not in df_orig_with_bad.columns:
    df_orig = df_orig_with_bad
else:
    series_bad = pd.isna(df_orig_with_bad['Linearization C'])
    IPython.display.display(df_orig_with_bad[series_bad])
    df_orig = df_orig_with_bad[~series_bad]

,Date,Scenario ID,Vehicle ID,Vehicle type,Cycle distance (m),No. of completed missions,Total distance traveled (m),No. of stops,No. of forcing events,No. of violations,No. of critical sections,No. of near-misses,No. of collisions,Is blocked,Total waiting time (s),Maximum waiting time (s),Total time (s),Maximum acceleration (m/s^2),Maximum speed (m/s),Average speed (m/s),Scenario,Current datetime,Time passed (real),Time passed (sim.),isCanPassFirstActive,isRacingThroughCrossroadAllowed,probabilitySlowingDownForHuman,Human V0,Collision events,Vehicle size (m),v_current,v_max,can pass first,violation of priorities,moving slowly,improper parking,cautious mode,rerouting at parked / slow,moving backwards,change of priorities,stops,violations,near misses,collisions,"traveled total, m",no. missions,blocked,"position (x, y), m","traveled, m",path index,no. poses,CP (index),"posTo Slow, m","distance ToCP, m",status,future missions,Linearization A,Linearization B,Linearization C,Linearization D1,Linearization D2,Linearization D3,Linearization D0
0,20241226_151426,"map-generator/generated-maps/2024-11-28_13:17:39_with_bridges/scenario7-1.json, baseline, seed 1, probabilityForcingForHuman 0; AUTOMATED_FIRST",0,HumanDrivenVehicle,454.75,5,2384.0,12,0,0,69,0,3,0,605.6,259.2,1598.6,0.3,2.6,1.5,"map-generator/generated-maps/2024-11-28_13:17:39_with_bridges/scenario7-1.json, baseline, seed 1, probabilityForcingForHuman 0; AUTOMATED_FIRST",2024-12-26 15:23:51,0:09:24,0:26:38 (x2.8),"hum=true, aut=false",True,0.0,0 forcing events,"0 minor, 4 major (0:08:51: V0[114]-V3[737], 0:10:18: V2[504]-V3[843], 0:14:19: V0[25]-V2[500], 0:26:21: V0[67]-V3[786])","[length=10.0, width=10.0, safe distance: front=5.0, back=5.0, left=1.0, right=1.0]",2.6,2.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0,3,2383.8,5,NaN,"(466.3, 139.2)",112.6,111,454,NaN,447.0,NaN,DRIVING,"2: [455, 454]",NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,20241226_151426,"map-generator/generated-maps/2024-11-28_13:17:39_with_bridges/scenario7-1.json, baseline, seed 1, probabilityForcingForHuman 0; AUTOMATED_FIRST",1,AutonomousVehicle,436.61,8,7768.3,9,0,0,39,0,0,0,15.4,10.7,1598.6,0.3,5.6,4.9,"map-generator/generated-maps/2024-11-28_13:17:39_with_bridges/scenario7-1.json, baseline, seed 1, probabilityForcingForHuman 0; AUTOMATED_FIRST",2024-12-26 15:23:51,0:09:24,0:26:38 (x2.8),"hum=true, aut=false",True,0.0,0 forcing events,"0 minor, 4 major (0:08:51: V0[114]-V3[737], 0:10:18: V2[504]-V3[843], 0:14:19: V0[25]-V2[500], 0:26:21: V0[67]-V3[786])","[length=10.0, width=10.0, safe distance: front=5.0, back=5.0, left=1.0, right=1.0]",5.6,5.6,NaN,NaN,NaN,NaN,NaN,0 / 0,NaN,NaN,NaN,NaN,0,0,7767.8,8,NaN,"(353.0, 484.6)",796.4,793,871,NaN,840.7,NaN,DRIVING,1: [871],NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,20241226_151426,"map-generator/generated-maps/2024-11-28_13:17:39_with_bridges/scenario7-1.json, baseline, seed 1, probabilityForcingForHuman 0; AUTOMATED_FIRST",2,AutonomousVehicle,1354.80,6,6969.9,11,0,0,59,0,2,0,149.5,80.1,1598.6,0.3,5.6,4.4,"map-generator/generated-maps/2024-11-28_13:17:39_with_bridges/scenario7-1.json, baseline, seed 1, probabilityForcingForHuman 0; AUTOMATED_FIRST",2024-12-26 15:23:51,0:09:24,0:26:38 (x2.8),"hum=true, aut=false",True,0.0,0 forcing events,"0 minor, 4 major (0:08:51: V0[114]-V3[737], 0:10:18: V2[504]-V3[843], 0:14:19: V0[25]-V2[500], 0:26:21: V0[67]-V3[786])","[length=10.0, width=10.0, safe distance: front=5.0, back=5.0, left=1.0, right=1.0]",5.6,5.6,NaN,NaN,NaN,NaN,NaN,4 / 0,NaN,NaN,NaN,NaN,0,2,6969.3,6,NaN,"(389.5, 367.8)",733.1,733,1242,841,809.4,109.3,DRIVING,1: [1135],NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,20241226_151426,"map-generator/generated-maps/2024-11-28_13:17:39_with_bridges/scenario7-1.json, baseline, seed 1, probabilityForcingForHuman 0; AUTOMATED_FIRST",3,AutonomousVehicle,2302.95,4,6764.8,11,0,0,57,0,3,0,224.2,143.3,1598.6,0.3,5.6,4.2,"map-generator/generated-maps/2024-11-28_13:17:39_with_bridges/scenario7-1.json, baseline, seed 1, probabilityForcingForHuman 0; AUTOMATED_FIRST",2024

In [5]:
def normalize_linearization(lin):
    if lin is None:
        return None
    return tuple(
        np.interp(
            np.linspace(0, 1, 100),
            np.linspace(0, 1, len(lin)), 
            lin
        )
    )


# Function to combine tuples into a tuple of small tuples
def combine_tuples(cols, row):
    # Extract the relevant columns from the row
    selected = [row[col] for col in cols]
    assert len(selected) > 1
    assert sum(c is None for c in selected) == 1, (cols, row)  # .to_dict())
    # Replace None with tuples of Nones based on the size of the first non-None tuple
    tuples = [col 
              if col is not None else 
              (None,) * len(next(c for c in selected if c is not None)) 
              for col in selected]
    # Combine using zip
    return tuple(zip(*tuples))


def normalize_df(df_orig):
    df_id = df_orig['Scenario ID'].str.split(r'[;,] ', expand=True)
    df_id.columns = [
        'filename', 'Coordination strategy', 'string_seed', 'string_probabilityForcingForHuman', 'heuristic',
    ]
    df_id = pd.concat([
        df_id,
        df_id['filename'].str.extract(r'(?P<dir_map>[^/]+)/(?P<basename_scenario>[^/]+)[.]json$', expand=True),
        df_id['filename'].str.extract(
            r'/scenario(?P<i_map>\d+)-(?P<i_locations>\d+)[.]json$', expand=True
        ).astype(int),
        df_id['string_probabilityForcingForHuman'].str.extract(
            r'^probabilityForcingForHuman (?P<probabilityForcingForHuman>[\d.]+)$', expand=True
        ).astype(float),
    ], axis=1).rename(columns={'i_locations': 'Positions variant'})
    df_id['filename_screenshot'] = "../map-generator/generated-maps/" + df_id['dir_map'] + '/screenshots/' + df_id['basename_scenario'] + '.png'
    df_id['are_bridges'] = df_id['dir_map'].str.contains('with_bridges')
    df_id['configuration'] = df_id[
        ['i_map', 'are_bridges', 'Positions variant']].agg(
        lambda r: f'map {r['i_map']}, {"with" if r['are_bridges'] else "without"} bridges, pos.var. {r['Positions variant']}', 
        axis=1
    )
    #df_id = df_id[df_id['i_map'] == I_MAP]
    
    df = pd.concat(
        [
            df_id[['i_map', 'are_bridges', 'Positions variant', 'configuration', 
                   'Coordination strategy', 'probabilityForcingForHuman', 'filename_screenshot']],
            df_orig['isCanPassFirstActive'].str.extract(
                r'^hum=(?P<isCanPassFirstHum>false|true), aut=(?P<isCanPassFirstAut>false|true)$', expand=True
            ).apply(lambda col: col == 'true'),
            df_orig['rerouting at parked / slow'].str.extract(
                r'^(?P<reroutingsAtParked>-|\d+) / (?P<reroutingsAtSlow>-|\d+)$', expand=True
            ).apply(lambda col: col.map(lambda x: np.nan if pd.isna(x) or x == '-' else int(x))).astype('Int64'),
            df_orig,
        ],
        axis=1
    )
    df.sort_values(
        ['i_map', 'are_bridges', 'Positions variant', 'Vehicle ID'], 
        ascending=[True, False, True, True],
        inplace=True,
    )
    
    postfix_nonnormalized = ' (non-normalized)'
    col_lin_d = 'Linearization D'
    pairs_lin_d: List[Tuple[int, str]] = []
    for col in df.columns:
        if col.startswith('Linearization'):
            series = df[col].apply(lambda x: None if pd.isna(x) else tuple(map(float, x.split())))
            #print(col, series)
            
            col_nonnormalized = col + postfix_nonnormalized
            df[col_nonnormalized] = series
            if col.startswith(col_lin_d):
                id_vehicle = int(col[len(col_lin_d):]) 
                pairs_lin_d.append((id_vehicle, col_nonnormalized))
            
            df[col] = series.apply(normalize_linearization)
            
    # Create new column
    if pairs_lin_d:
        cols_lin_d = [col for _, col in sorted(pairs_lin_d)]
        df[col_lin_d + postfix_nonnormalized] = df.apply(lambda row: combine_tuples(cols_lin_d, row), axis=1)
    
    return df


df_orig_norm = normalize_df(df_orig)
configuration_to_filename_screenshot = {row['configuration']: row['filename_screenshot'] for _, row in df_orig_norm.iterrows()}
df_orig_norm

,i_map,are_bridges,Positions variant,configuration,Coordination strategy,probabilityForcingForHuman,filename_screenshot,isCanPassFirstHum,isCanPassFirstAut,reroutingsAtParked,reroutingsAtSlow,Date,Scenario ID,Vehicle ID,Vehicle type,Cycle distance (m),No. of completed missions,Total distance traveled (m),No. of stops,No. of forcing events,No. of violations,No. of critical sections,No. of near-misses,No. of collisions,Is blocked,Total waiting time (s),Maximum waiting time (s),Total time (s),Maximum acceleration (m/s^2),Maximum speed (m/s),Average speed (m/s),Scenario,Current datetime,Time passed (real),Time passed (sim.),isCanPassFirstActive,isRacingThroughCrossroadAllowed,probabilitySlowingDownForHuman,Human V0,Collision events,Vehicle size (m),v_current,v_max,can pass first,violation of priorities,moving slowly,improper parking,cautious mode,rerouting at parked / slow,moving backwards,change of priorities,stops,violations,near misses,collisions,"traveled total, m",no. missions,blocked,"position (x, y), m","traveled, m",path index,no. poses,CP (index),"posTo Slow, m","distance ToCP, m",status,future missions,Linearization A,Linearization B,Linearization C,Linearization D1,Linearization D2,Linearization D3,Linearization D0,Linearization A (non-normalized),Linearization B (non-normalized),Linearization C (non-normalized),Linearization D1 (non-normalized),Linearization D2 (non-normalized),Linearization D3 (non-normalized),Linearization D0 (non-normalized),Linearization D (non-normalized)
4,1,True,1,"map 1, with bridges, pos.var. 1",baseline,0.0,../map-generator/generated-maps/2024-11-28_13:17:39_with_bridges/screenshots/scenario1-1.png,True,False,<NA>,<NA>,20241226_151429,"map-generator/generated-maps/2024-11-28_13:17:39_with_bridges/scenario1-1.json, baseline, seed 1, probabilityForcingForHuman 0; AUTOMATED_FIRST",0,HumanDrivenVehicle,778.12,4,3665.9,18,0,0,76,0,0,0,289.9,64.1,1799.9,0.3,2.6,2.0,"map-generator/generated-maps/2024-11-28_13:17:39_with_bridges/scenario1-1.json, baseline, seed 1, probabilityForcingForHuman 0; AUTOMATED_FIRST",2024-12-26 15:21:06,0:06:37,0:29:59 (x4.5),"hum=true, aut=false",True,0.0,0 forcing events,"0 minor, 0 major","[length=10.0, width=10.0, safe distance: front=5.0, back=5.0, left=1.0, right=1.0]",2.6,2.6,no,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0,0,3664.9,4,NaN,"(273.3, 146.2)",554.7,555,778,581,573.8,28.2,DRIVING,"2: [777, 778]","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.001318, 0.001318, 0.001318, 0.001318, 0.001318, 0.001318, 0.001318, 0.001318, 0.001318, 0.001318, 0.001318, 0.001318, 0.001318, 0.001318, 0.001...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.305867, 0.305867, 0.305867, 0.305867, 0.305867, 0.305867, 0.305867, 0.305867, 0.305867, 0.305867, 0.305867, 0.305867, 0.305867, 0.305867, 0.305...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.305867, 0.305867, 0.305867, 0.305867, 0.305867, 0.305867, 0.305867, 0.305867, 0.305867, 0.305867, 0.305867, 0.305867, 0.305867, 0.305867, 0.305...",None,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.

In [6]:
df_lin = None if RUNNAME_LINEARIZATIONS is None else read_df_runname(RUNNAME_LINEARIZATIONS)
df_lin

In [7]:
df_lin_norm = None if df_lin is None else normalize_df(df_lin)
df_lin_norm

In [8]:
def add_linearizations(df_orig_norm, df_lin_norm):
    if df_lin_norm is None:
        return df_orig_norm

    # Get columns that start with 'Linearization'
    linearization_cols = [col for col in df_lin_norm.columns if col.startswith('Linearization')]
    assert linearization_cols
    
    # Specified keys for merging
    keys = ['i_map', 'are_bridges', 'Positions variant', 'Vehicle ID']
    
    # Check if keys exist in both dataframes
    missing_keys = [key for key in keys
                    if key not in df_lin_norm.columns
                    or key not in df_orig_norm.columns]
    if missing_keys:
        raise KeyError(f"Missing key columns in either DataFrame: {missing_keys}")
    
    # Perform an inner merge to ensure all data from df_lin is matched and not missing
    merged_df = pd.merge(df_orig_norm, df_lin_norm[keys + linearization_cols], on=keys, how='inner')
    
    # Check if the merged DataFrame has any missing data from df_lin
    #if merged_df[linearization_cols].isnull().any().any():
    #    raise ValueError("Some linearization is missing in the merged DataFrame.")
    
    return merged_df


df_all = add_linearizations(df_orig_norm, df_lin_norm)
if I_MAP == 1:
    df_all.to_csv(f'{DIRECTORY_DATA}/df_all.csv')
df_all

,i_map,are_bridges,Positions variant,configuration,Coordination strategy,probabilityForcingForHuman,filename_screenshot,isCanPassFirstHum,isCanPassFirstAut,reroutingsAtParked,reroutingsAtSlow,Date,Scenario ID,Vehicle ID,Vehicle type,Cycle distance (m),No. of completed missions,Total distance traveled (m),No. of stops,No. of forcing events,No. of violations,No. of critical sections,No. of near-misses,No. of collisions,Is blocked,Total waiting time (s),Maximum waiting time (s),Total time (s),Maximum acceleration (m/s^2),Maximum speed (m/s),Average speed (m/s),Scenario,Current datetime,Time passed (real),Time passed (sim.),isCanPassFirstActive,isRacingThroughCrossroadAllowed,probabilitySlowingDownForHuman,Human V0,Collision events,Vehicle size (m),v_current,v_max,can pass first,violation of priorities,moving slowly,improper parking,cautious mode,rerouting at parked / slow,moving backwards,change of priorities,stops,violations,near misses,collisions,"traveled total, m",no. missions,blocked,"position (x, y), m","traveled, m",path index,no. poses,CP (index),"posTo Slow, m","distance ToCP, m",status,future missions,Linearization A,Linearization B,Linearization C,Linearization D1,Linearization D2,Linearization D3,Linearization D0,Linearization A (non-normalized),Linearization B (non-normalized),Linearization C (non-normalized),Linearization D1 (non-normalized),Linearization D2 (non-normalized),Linearization D3 (non-normalized),Linearization D0 (non-normalized),Linearization D (non-normalized)
4,1,True,1,"map 1, with bridges, pos.var. 1",baseline,0.0,../map-generator/generated-maps/2024-11-28_13:17:39_with_bridges/screenshots/scenario1-1.png,True,False,<NA>,<NA>,20241226_151429,"map-generator/generated-maps/2024-11-28_13:17:39_with_bridges/scenario1-1.json, baseline, seed 1, probabilityForcingForHuman 0; AUTOMATED_FIRST",0,HumanDrivenVehicle,778.12,4,3665.9,18,0,0,76,0,0,0,289.9,64.1,1799.9,0.3,2.6,2.0,"map-generator/generated-maps/2024-11-28_13:17:39_with_bridges/scenario1-1.json, baseline, seed 1, probabilityForcingForHuman 0; AUTOMATED_FIRST",2024-12-26 15:21:06,0:06:37,0:29:59 (x4.5),"hum=true, aut=false",True,0.0,0 forcing events,"0 minor, 0 major","[length=10.0, width=10.0, safe distance: front=5.0, back=5.0, left=1.0, right=1.0]",2.6,2.6,no,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0,0,3664.9,4,NaN,"(273.3, 146.2)",554.7,555,778,581,573.8,28.2,DRIVING,"2: [777, 778]","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.001318, 0.001318, 0.001318, 0.001318, 0.001318, 0.001318, 0.001318, 0.001318, 0.001318, 0.001318, 0.001318, 0.001318, 0.001318, 0.001318, 0.001...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.305867, 0.305867, 0.305867, 0.305867, 0.305867, 0.305867, 0.305867, 0.305867, 0.305867, 0.305867, 0.305867, 0.305867, 0.305867, 0.305867, 0.305...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.305867, 0.305867, 0.305867, 0.305867, 0.305867, 0.305867, 0.305867, 0.305867, 0.305867, 0.305867, 0.305867, 0.305867, 0.305867, 0.305867, 0.305...",None,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.

In [12]:
df_all[~pd.isna(df_all['blocked'])]

,i_map,are_bridges,Positions variant,configuration,Coordination strategy,probabilityForcingForHuman,filename_screenshot,isCanPassFirstHum,isCanPassFirstAut,reroutingsAtParked,reroutingsAtSlow,Date,Scenario ID,Vehicle ID,Vehicle type,Cycle distance (m),No. of completed missions,Total distance traveled (m),No. of stops,No. of forcing events,No. of violations,No. of critical sections,No. of near-misses,No. of collisions,Is blocked,Total waiting time (s),Maximum waiting time (s),Total time (s),Maximum acceleration (m/s^2),Maximum speed (m/s),Average speed (m/s),Scenario,Current datetime,Time passed (real),Time passed (sim.),isCanPassFirstActive,isRacingThroughCrossroadAllowed,probabilitySlowingDownForHuman,Human V0,Collision events,Vehicle size (m),v_current,v_max,can pass first,violation of priorities,moving slowly,improper parking,cautious mode,rerouting at parked / slow,moving backwards,change of priorities,stops,violations,near misses,collisions,"traveled total, m",no. missions,blocked,"position (x, y), m","traveled, m",path index,no. poses,CP (index),"posTo Slow, m","distance ToCP, m",status,future missions,Linearization A,Linearization B,Linearization C,Linearization D1,Linearization D2,Linearization D3,Linearization D0,Linearization A (non-normalized),Linearization B (non-normalized),Linearization C (non-normalized),Linearization D1 (non-normalized),Linearization D2 (non-normalized),Linearization D3 (non-normalized),Linearization D0 (non-normalized),Linearization D (non-normalized)
26,9,True,1,"map 9, with bridges, pos.var. 1",change of priorities,1.0,../map-generator/generated-maps/2024-11-28_13:17:39_with_bridges/screenshots/scenario9-1.png,True,False,5,0,20241226_152547,"map-generator/generated-maps/2024-11-28_13:17:39_with_bridges/scenario9-1.json, change of priorities, seed 1, probabilityForcingForHuman 1; AUTOMATED_FIRST",2,AutonomousVehicle,2125.74,5,4543.0,12,0,0,50,0,1,1,774.0,506.7,1799.9,0.3,5.6,2.5,"map-generator/generated-maps/2024-11-28_13:17:39_with_bridges/scenario9-1.json, change of priorities, seed 1, probabilityForcingForHuman 1; AUTOMATED_FIRST",2024-12-26 15:33:44,0:07:57,0:29:59 (x3.8),"hum=true, aut=false",True,0.0,5 forcing events,"0 minor, 2 major (0:02:40: V0[405]-V1[399], 0:09:23: V0[520]-V2[713])","[length=10.0, width=10.0, safe distance: front=5.0, back=5.0, left=1.0, right=1.0]",0.0,5.6,NaN,NaN,NaN,NaN,NaN,5 / 0,NaN,NaN,NaN,NaN,0,1,4543.0,5,0:08:26,"(64.5, 274.6)",429.5,428,430,565,414.8,NaN,STOP@CP,1: [1131],"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 3.0, 3.0, 3.0, 3.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.000934, 0.000934, 0.000934, 0.000934, 0.000934, 0.000934, 0.000934, 0.000934, 0.000934, 0.000934, 0.000934, 0.000934, 0.000934, 0.000934, 0.000934, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.879552, 0.879552, 0.879552, 0.879552, 0.879552, 0.879552, 0.879552, 0.879552, 0.879552, 0.879552, 0.879552, 0.879552, 0.879552, 0.879552, 0.879552, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.315152, 0.315152, ...",None,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.879552, 0.879552, 0.879552, 0.879552, 0.879552, 0.879552, 0.879552, 0.879552, 0.879552, 0.879552, 0.879552, 0.879552, 0.879552, 0.879552, 0.879552, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.182514, 0.182514, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0

# Filtering by `i_map`

In [9]:
df_map = df_all[df_all['i_map'] == I_MAP]
df_map

,i_map,are_bridges,Positions variant,configuration,Coordination strategy,probabilityForcingForHuman,filename_screenshot,isCanPassFirstHum,isCanPassFirstAut,reroutingsAtParked,reroutingsAtSlow,Date,Scenario ID,Vehicle ID,Vehicle type,Cycle distance (m),No. of completed missions,Total distance traveled (m),No. of stops,No. of forcing events,No. of violations,No. of critical sections,No. of near-misses,No. of collisions,Is blocked,Total waiting time (s),Maximum waiting time (s),Total time (s),Maximum acceleration (m/s^2),Maximum speed (m/s),Average speed (m/s),Scenario,Current datetime,Time passed (real),Time passed (sim.),isCanPassFirstActive,isRacingThroughCrossroadAllowed,probabilitySlowingDownForHuman,Human V0,Collision events,Vehicle size (m),v_current,v_max,can pass first,violation of priorities,moving slowly,improper parking,cautious mode,rerouting at parked / slow,moving backwards,change of priorities,stops,violations,near misses,collisions,"traveled total, m",no. missions,blocked,"position (x, y), m","traveled, m",path index,no. poses,CP (index),"posTo Slow, m","distance ToCP, m",status,future missions,Linearization A,Linearization B,Linearization C,Linearization D1,Linearization D2,Linearization D3,Linearization D0,Linearization A (non-normalized),Linearization B (non-normalized),Linearization C (non-normalized),Linearization D1 (non-normalized),Linearization D2 (non-normalized),Linearization D3 (non-normalized),Linearization D0 (non-normalized),Linearization D (non-normalized)
4,1,True,1,"map 1, with bridges, pos.var. 1",baseline,0.0,../map-generator/generated-maps/2024-11-28_13:17:39_with_bridges/screenshots/scenario1-1.png,True,False,<NA>,<NA>,20241226_151429,"map-generator/generated-maps/2024-11-28_13:17:39_with_bridges/scenario1-1.json, baseline, seed 1, probabilityForcingForHuman 0; AUTOMATED_FIRST",0,HumanDrivenVehicle,778.12,4,3665.9,18,0,0,76,0,0,0,289.9,64.1,1799.9,0.3,2.6,2.0,"map-generator/generated-maps/2024-11-28_13:17:39_with_bridges/scenario1-1.json, baseline, seed 1, probabilityForcingForHuman 0; AUTOMATED_FIRST",2024-12-26 15:21:06,0:06:37,0:29:59 (x4.5),"hum=true, aut=false",True,0.0,0 forcing events,"0 minor, 0 major","[length=10.0, width=10.0, safe distance: front=5.0, back=5.0, left=1.0, right=1.0]",2.6,2.6,no,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0,0,3664.9,4,NaN,"(273.3, 146.2)",554.7,555,778,581,573.8,28.2,DRIVING,"2: [777, 778]","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.001318, 0.001318, 0.001318, 0.001318, 0.001318, 0.001318, 0.001318, 0.001318, 0.001318, 0.001318, 0.001318, 0.001318, 0.001318, 0.001318, 0.001...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.305867, 0.305867, 0.305867, 0.305867, 0.305867, 0.305867, 0.305867, 0.305867, 0.305867, 0.305867, 0.305867, 0.305867, 0.305867, 0.305867, 0.305...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.305867, 0.305867, 0.305867, 0.305867, 0.305867, 0.305867, 0.305867, 0.305867, 0.305867, 0.305867, 0.305867, 0.305867, 0.305867, 0.305867, 0.305...",None,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.

In [10]:
def display_groups(groups):
    for key, df in groups:
        print(key)
        cols = [col 
                for col in ['configuration', 'Coordination strategy', 'Vehicle ID', 'Is blocked'] 
                if col in df.columns]
        IPython.display.display(df[cols].sort_values(by=cols))
        break


def make_indexes_blocked_nonblocked(cols, *, is_verbose=False):
    df = df_map[df_map['Vehicle type'] != 'HumanDrivenVehicle']
    groups_blocks = df.groupby(cols, sort=False)
    if is_verbose:
        display_groups(groups_blocks)
    series_blocks = groups_blocks['Is blocked'].sum()
    
    index_blocked = series_blocks[series_blocks != 0].index
    index_nonblocked = series_blocks[series_blocks == 0].index
    return index_blocked, index_nonblocked


make_indexes_blocked_nonblocked(['are_bridges', 'Positions variant', 'Coordination strategy'], is_verbose=True)

(True, 1, 'baseline')


,configuration,Coordination strategy,Vehicle ID,Is blocked
5,"map 1, with bridges, pos.var. 1",baseline,1,0
6,"map 1, with bridges, pos.var. 1",baseline,2,0
7,"map 1, with bridges, pos.var. 1",baseline,3,0


(MultiIndex([], names=['are_bridges', 'Positions variant', 'Coordination strategy']),
 MultiIndex([( True, 1,             'baseline'),
             ( True, 1, 'change of priorities'),
             ( True, 1,                'stops'),
             (False, 1,             'baseline')],
            names=['are_bridges', 'Positions variant', 'Coordination strategy']))

# Main plots

In [11]:
def save_and_show(fig, basename):  # to avoid inlining large image data into the notebook file
    filename = f'{DIRECTORY_DATA}/{basename}.png'
    fig.savefig(filename)
    
    # The `random` is because of https://stackoverflow.com/a/43640705.
    IPython.display.display(IPython.display.HTML(f'<img src="{filename}?{random.random()}" alt="{basename}" />'))
    
    plt.close(fig)
    
    return filename


def same_value(series):
    assert series.nunique(dropna=False) == 1, series
    return series.iloc[0]
    

def find_misbehaviors(df_nonbaseline):
    misbehaviors = []
    
    probabilityForcingForHuman = same_value(df_nonbaseline['probabilityForcingForHuman'])
    if probabilityForcingForHuman > 0.0:
        misbehaviors.append(f'violation of priorities ({"random" if probabilityForcingForHuman < 1.0 else "constant"})')
        
    isCanPassFirstActive = same_value(df_nonbaseline['isCanPassFirstActive'])
    if isCanPassFirstActive.startswith('hum=true, '):
        misbehaviors.append('can pass first')
    elif isCanPassFirstActive.startswith('hum=false, '):
        pass
    else:
        raise ValueError(isCanPassFirstActive)
    
    if 'probabilitySlowingDownForHuman' in df_nonbaseline.columns:
        probabilitySlowingDownForHuman = float(same_value(df_nonbaseline['probabilitySlowingDownForHuman']))
        if probabilitySlowingDownForHuman > 0.0:
            misbehaviors.append(f'moving slowly ({"random" if probabilitySlowingDownForHuman < 1.0 else "constant"})')
    
    return misbehaviors


def make_subplots(ncols):
    return plt.subplots(1, ncols, figsize=(20, 6), sharey=True, squeeze=False)
    

def plot_title(df, *, title2):
    positions = df.index.get_level_values('Positions variant').unique()
    fig, axes = make_subplots(len(positions))
    default_fig_width, default_fig_height = fig.get_size_inches()
    plt.close(fig)
    
    fig = plt.figure(figsize=(default_fig_width, 1))
    
    misbehaviors = find_misbehaviors(df[df['Coordination strategy'] != 'baseline'])
    title3 = 'Human (mis)behaviour actions: ' + ('none' if not misbehaviors else ', '.join(misbehaviors))    
    
    title = f'{title2}\n{title3}'
    fig.suptitle(title, fontsize=16)
    
    fig.tight_layout()
    title1 = '.title'
    filename_png = save_and_show(fig, f'{title1}: {title2}')
    
    return filename_png
    
    
class Formula:
    def __init__(self, label, expression=None):
        self.label = label
        self.expression = expression if expression is not None else label
        
    def __str__(self):
        return self.label.replace('`', '')
    
    def __repr__(self):
        return f'<Formula: {self.label!r}>'
    
    def apply(self, df):
        if isinstance(self.expression, str):
            return df.eval(self.expression)
        return df.apply(self.expression, axis=1)
    
    
def add_categorical_classes(ax, n):
    """
    Add N classes to the x-axis with N-1 ticks in each class, excluding the class index.

    Parameters:
    ax (matplotlib.axes.Axes): The axis to modify.
    n (int): The number of classes (vehicles).
    """
    # Generate tick positions and labels for the primary x-axis
    primary_ticks = []
    primary_labels = []
    for i in range(n):
        start = i * (n - 1)
        ticks = [k for k in range(n) if k != i]  # Exclude the class index
        primary_ticks.extend(start + np.arange(len(ticks)))
        primary_labels.extend([str(tick) for tick in ticks])

    ax.set_xticks(primary_ticks)
    ax.set_xticklabels(primary_labels)

    # Add secondary x-axis for class annotations
    sec = ax.secondary_xaxis(location=0)

    # Define class boundaries and labels
    class_centers = [(i * (n - 1) + (n - 2) / 2) for i in range(n)]
    class_labels = [f"\n\n{i}" for i in range(n)]
    sec.set_xticks(class_centers, labels=class_labels)
    sec.tick_params('x', length=0)

    # Add another secondary x-axis for boundaries between classes
    sec2 = ax.secondary_xaxis(location=0)
    class_boundaries = np.arange(-0.5, n * (n - 1), n - 1)
    sec2.set_xticks(class_boundaries, labels=[])
    sec2.tick_params('x', length=40, width=1.5)

    # Adjust the axis limits
    ax.set_xlim(-0.5, n * (n - 1) - 0.5)
    
    
def plot_vertical_heatmap(ax, max_length, df, column):    
    cells = df[column]
    
    is_normalized = not column.endswith(' (non-normalized)')

    # Expand each list to have the same length
    expanded_data = []
    is_matrix = False
    for cell in cells:
        assert isinstance(cell, tuple)
        assert cell
        vectors: List[Tuple[float]] = []
        if isinstance(cell[0], float):
            vectors.append(cell)
        else:
            assert isinstance(cell[0], tuple)
            is_matrix = True
            for j in range(len(cell[0])):
                vector = tuple(cell[i][j] for i in range(len(cell)))
                assert vector
                assert all(type(x) == type(vector[0]) for x in vector)
                if vector[0] is not None:
                    assert isinstance(vector[0], float)
                    vectors.append(vector)
        
        for vector in vectors:
            k_extra = max_length - len(cell)
            if is_normalized:
                assert k_extra == 0
            expanded_data.append(vector + (np.nan,) * k_extra)

    # Convert the expanded data to a 2D NumPy array
    data_array = np.array(expanded_data)

    # Transpose the data array to have heatmaps side-by-side (columns represent each row)
    data_array_transposed = data_array.T

    # Plot the heatmap - each column is a vertical slice now (grayscale, 0 is white, max is black)
    im = ax.imshow(data_array_transposed, aspect='auto', cmap='gray_r', interpolation='nearest')

    # Set labels for better readability
    labels_ids = list(df.index.get_level_values("Vehicle ID"))
    if not is_matrix:
        ax.set_xlabel('Vehicle ID')
        ax.set_xticks(np.arange(len(df.index)))
        ax.set_xticklabels(labels_ids)
    else:
        n_vehicles = len(labels_ids)
        assert labels_ids == list(range(n_vehicles))
        add_categorical_classes(ax, n_vehicles)
        ax.set_xlabel('\n\nVehicle ID')
    
    if is_normalized:
        ax.set_ylabel('Stage along the path')
        y_ticks = np.linspace(0, max_length - 1, 5)  # Set 5 evenly spaced ticks from 0 to max_length - 1
        ax.set_yticks(y_ticks)  # Apply these y-tick positions
        ax.set_yticklabels([f'{int((tick / (max_length - 1)) * 100)}%' for tick in y_ticks])  # Label ticks from 0% to 100%
    else:    
        ax.set_ylabel('Meters of the path')
        
        # Generate mask for NaN values
        nan_mask = np.isnan(data_array_transposed)
        
        # Adding rectangles around non-NaN regions for each column
        for col in range(data_array_transposed.shape[1]):
            nan_rows = np.where(nan_mask[:, col])[0]
            
            # Determine the start and end of non-NaN regions
            if len(nan_rows) == 0:
                # No NaN values in the column; entire column is non-NaN
                start_row = 0
                end_row = data_array_transposed.shape[0]
            else:
                # If there are NaNs, find the first occurrence
                start_row = 0
                end_row = nan_rows[0]
            
            # Draw rectangle around non-NaN values
            rect = Rectangle((col - 0.5, start_row - 0.5), 1, end_row - start_row, edgecolor='black', facecolor='none', linewidth=2)
            ax.add_patch(rect)
    
    # Return the image object for colorbar usage
    return im


def plot_lin(df, *, title2, column):
    #IPython.display.display(df)
    
    # Plot configuration
    positions = df.index.get_level_values('Positions variant').unique()    
    fig, axes = make_subplots(len(positions))
    axes: list[list[Axes]]

    # Find the maximum length of rows
    max_length = max(
        max(df.loc[position, column].apply(len))
        for position in positions
    )
    
    images = []
    for i, position in enumerate(positions):
        ax: Axes = axes[0][i]
        df_pos = df.loc[position]
        images.append(plot_vertical_heatmap(ax, max_length, df_pos, column=column))
        ax.set_title(f'Position {position}')
    
    title1 = f'Paths segmentation based on CS density ({column})'
    column2description = {
        'Linearization': 'From the full simulation. Same as Linearization A.', 
        'Linearization A': 'From the short simulation. CS count: +1 for each CS.', 
        'Linearization B': "From the short simulation. For each CS: +1 / other's path length.", 
        'Linearization C': "From the short simulation. For each CS: + CS length / other's path length.", 
    }
    description = column2description.get(column)
    
    title = title1
    if description is not None:
        title += f'\n({description})'
    fig.suptitle(title, fontsize=16)
    
    fig.tight_layout()
    fig.subplots_adjust(top=0.85)
    
    cax = fig.add_axes([0.85, 0.9, 0.1, 0.03])  # Adjust values to position the colorbar correctly
    fig.colorbar(
        images[0], cax=cax, label='Density',
        orientation='horizontal', location='top',
    )
    
    filename_png = save_and_show(fig, f'{title1}: {title2}')
    
    #print(id(df))
    #IPython.display.display(df)
    return filename_png, {}
    
    
def plot_aut_hum(df, *, title2, dfs_y1, dfs_y2, index_blocked_pos_strat, mode):
    #IPython.display.display(df)
    
    # Plot configuration
    positions = df.index.get_level_values('Positions variant').unique()    
    fig, axes = make_subplots(len(positions))
    axes: list[list[Axes]]
    bar_width = 0.4
 
    strategy2label = {'baseline': 'baseline\n(no human effect)', 'stops': 'stops (local)'}        

    formulas_y2_aut = [Formula('`No. of collisions`'), Formula('`No. of near-misses`')]
    colors_y2_aut = ['red', 'yellow']
    
    formulas_y2_hum = [Formula('`No. of violations`')] + formulas_y2_aut
    colors_y2_hum = ['black'] + colors_y2_aut
    
    formulas_y2_cmp = [Formula(f'`{col}` / `No. of violations`', 
                               lambda row, _col=col: row[_col] / row['No. of violations'] if row['No. of violations'] > 0 else 0)
                       for col in ('No. of collisions', 'No. of near-misses')]
    colors_y2_cmp = ['red', 'yellow']
    
    if mode.startswith('aut_') or mode == 'hum':
        if mode == 'aut_missions':
            column_y1 = 'No. of completed missions'
            color_y1 = 'tab:green'
        elif mode == 'aut_reroutings':
            column_y1 = 'reroutingsAtParked'
            color_y1 = '#9467bd'  # violet
        else:
            column_y1 = 'Total distance traveled (m)'
            color_y1 = 'tab:blue'
        
        if mode.startswith('aut_'):
            if mode == 'aut_reroutings':
                formulas_y2 = []
                colors_y2 = []
            else:
                formulas_y2 = formulas_y2_aut
                colors_y2 = colors_y2_aut
            id_vehicle_max = df['Vehicle ID'].max()
            title1 = f'Automated vehicles (summarised for AV1-AV{id_vehicle_max})' 
        else:
            assert mode == 'hum'
            formulas_y2 = formulas_y2_hum
            colors_y2 = colors_y2_hum
            title1 = 'Human-driven vehicle'
            
    elif mode == 'cmp':
        column_y1 = None
        color_y1 = None
        
        formulas_y2 = formulas_y2_cmp
        colors_y2 = colors_y2_cmp
        title1 = 'Collisions rate'
        
    else:
        raise ValueError(f'Unknown mode {mode}')
    
    # Get the global max values for consistent y-axis scaling
    if column_y1 is not None:
        y1_max = max(dfx[column_y1].max() for dfx in dfs_y1)
        y1_lim = y1_max * 1.1
        
    if mode == 'cmp':
        y2_lim = 2.0
    elif not formulas_y2:
        y2_lim = None
    else:
        y2_maxes = [formula.apply(dfx).max()
                    for dfx in dfs_y2
                    for formula in (formulas_y2 if mode == 'cmp' else set(formulas_y2_aut + formulas_y2_hum))]
        y2_max = max(y2_maxes)
        y2_lim = y2_max * 1.1
    
    pos2metstrat2value = {}
    handles = []
    
    # Iterate through each Positions variant
    for i, position in enumerate(positions):
        ax: Axes = axes[0][i]
        df_pos = df.loc[position]
        pos2metstrat2value[position] = metstrat2value = {}
        
        def add_to_metstrat2value(metric, series):
            for strategy, value in series.items():
                metstrat2value[metric, strategy] = value
        
        # Bar positions for each Coordination strategy
        strategies = df_pos.index.get_level_values('Coordination strategy') 
        x_positions = np.arange(len(strategies))
        ax.set_xlabel('Coordination strategy')
        
        # Plot bars
        handles.clear()
        if column_y1 is None:
            ax.set_yticks([])
        else:
            if mode == 'aut_missions':
                add_to_metstrat2value(column_y1, df_pos[column_y1])
            handles += [ax.bar(x_positions, df_pos[column_y1], width=bar_width, label=column_y1, color=color_y1)[0]]
            ax.set_ylabel(column_y1, color=color_y1)
            ax.tick_params(axis='y', labelcolor=color_y1)
            ax.set_ylim(0, y1_lim)
            
        if y2_lim is not None:
            # Create a secondary axis for the points
            ax_right = ax.twinx()
            
            # Plot points
            label2series = {str(formula): formula.apply(df_pos)
                            for formula in formulas_y2}
            for (label, series), color in zip(label2series.items(), colors_y2):
                if mode == 'cmp':
                    add_to_metstrat2value(label, series)
                handles.append(
                    ax_right.plot(
                        x_positions, series, label=label, marker='o', linestyle='', color=color
                    )[0]
                )
            ax_right.tick_params(axis='y', labelcolor='black')
            ax_right.set_ylim(0, y2_lim)
        
        # Add labels, grid, and title for each section
        ax.set_xticks(x_positions)
        #index_blocked_pos_strat
        ax.set_xticklabels([strategy2label.get(s, s) for s in strategies], rotation=60, ha='right')
        for xtick, strategy in zip(ax.get_xticklabels(), strategies):
            if not index_blocked_pos_strat[
                (index_blocked_pos_strat.get_level_values('Positions variant') == position) &
                (index_blocked_pos_strat.get_level_values('Coordination strategy') == strategy)
            ].empty:
                xtick.set_color('orange')
        ax.set_title(f'Position {position}')
        ax.grid(axis='y')
        
    fig.suptitle(title1, fontsize=16)
    
    labels = [str(x) for x in [column_y1, *formulas_y2] if x is not None]
    fig.legend(handles=handles, labels=labels, ncol=len(handles), loc='upper right')
    
    fig.tight_layout()
    filename_png = save_and_show(fig, f'{title1}: {title2} ({mode})')
    
    #print(id(df))
    #IPython.display.display(df)    
    return filename_png, pos2metstrat2value


def plot_all(df_map):
    key2df = {}
    for are_bridges in True, False:
        for is_aut in None, True, False:
            dfx = df_map[df_map['are_bridges'] == are_bridges]
            #dfx = dfx[dfx['configuration'].isin(index_nonblocked.get_level_values('configuration'))]
            if is_aut is None:
                dfx = dfx.groupby(['Positions variant', 'Vehicle ID']).agg({
                    **{
                        col: same_value 
                        for col in dfx.columns 
                        if col.startswith('Linearization')
                        and col not in ('Linearization', 'Linearization (non-normalized)')  # because old data is broken
                    },
                })
            else:
                dfx = dfx[dfx['Vehicle type'] == ('AutonomousVehicle' if is_aut else 'HumanDrivenVehicle')]
                dfx = dfx.groupby(['Positions variant', 'Coordination strategy']).agg({
                    **{col: 'sum' for col in dfx.columns},
                    **{col: same_value for col in (
                        'Positions variant', 'Coordination strategy', 'configuration',
                        'probabilityForcingForHuman', 'isCanPassFirstActive', 'probabilitySlowingDownForHuman',
                    ) if col in dfx.columns},
                    **{col: 'max' for col in ('Vehicle ID', )},
                })
                
            key2df[are_bridges, is_aut] = dfx          
            
    index_blocked, index_nonblocked = make_indexes_blocked_nonblocked(
        ['are_bridges', 'Positions variant', 'Coordination strategy']
    )
    are_bridges_to_plotdicts = {}
    
    for are_bridges in True, False:
        filenames_png = [
            plot_title(
                key2df[are_bridges, True],
                title2=f"Map {I_MAP} ({'with' if are_bridges else 'without'} bridges)",
            )
        ]
        are_bridges_to_plotdicts[are_bridges] = plotdicts = []
        
        cols_lin = [
            col 
            for col in df_map 
            if re.match(
                r'''
                Linearization 
                (?:
                    [ ]
                    [ABCD]
                    # normalized 
                | 
                    [ ]
                    [CD]
                    [ ]
                    [(]non-normalized[)]
                )
                $
                ''', 
                col, 
                flags=re.VERBOSE,
            )
        ]
        
        modes = [
            *cols_lin,
            'aut_distance', 
            'aut_missions',  
            'hum', 
            'cmp',
            'aut_reroutings',
        ]
        
        for mode in modes:
            is_aut = mode.startswith('aut_')
            title2 = f"Map {I_MAP} ({'with' if are_bridges else 'without'} bridges)"
            
            filename_png, pos2metstrat2value = (
                plot_lin(
                    key2df[are_bridges, None],
                    title2=title2,
                    column=mode,
                )
                if mode.startswith('Linearization') else
                plot_aut_hum(
                    key2df[are_bridges, is_aut],
                    title2=title2, 
                    dfs_y1=[dfx for (_, is_aut_dfx), dfx in key2df.items() if is_aut_dfx == is_aut],
                    dfs_y2=[
                        key2df[are_bridges, is_aut_dfx] 
                        for is_aut_dfx in (
                            [is_aut] if mode == 'cmp' else [True, False]
                        )
                    ],
                    index_blocked_pos_strat=index_blocked[
                        index_blocked.get_level_values('are_bridges') == are_bridges
                    ],
                    mode=mode,
                )
            )
        
            filenames_png.append(filename_png)
            plotdicts.append(pos2metstrat2value)
            
        #IPython.display.display(dfx)
        filename_out_png = f'{DIRECTORY_DATA}/All: Map {I_MAP} (' + ('with' if are_bridges else 'without') + ' bridges).png'
        command_convert = ['convert', *filenames_png, '-append', filename_out_png]
        #print(f'{command_convert=}')
        assert len(set(filenames_png)) == len(filenames_png)
        subprocess.run(command_convert, check=True)
        #print(plotdicts)
        
    return merge_are_bridges_to_plotdicts(are_bridges_to_plotdicts)
    
        
def merge_are_bridges_to_plotdicts(are_bridges_to_plotdicts):
    merged_data = {}
    for are_bridges, plotdicts in are_bridges_to_plotdicts.items():
        for pos_dict in plotdicts:
            if are_bridges not in merged_data:
                merged_data[are_bridges] = {}
            for pos, metric_strat_dict in pos_dict.items():
                if pos not in merged_data[are_bridges]:
                    merged_data[are_bridges][pos] = {}
                merged_data[are_bridges][pos].update(metric_strat_dict)
                
    # Step 2: Extract all unique keys for indexing
    all_are_bridges = sorted(merged_data.keys())
    all_positions = sorted({pos for are_val in merged_data.values() for pos in are_val.keys()})
    all_pairs = [pair 
                 for are_val in merged_data.values() 
                 for pos_val in are_val.values() 
                 for pair in pos_val.keys()]
    
    unique_metrics = list({m: None for (m, s) in all_pairs})
    unique_strategies = list({s: None for (m, s) in all_pairs})
    
    # Step 3: Create MultiIndex for rows and columns
    row_index = pd.MultiIndex.from_product([[I_MAP], all_are_bridges, all_positions], names=["map", "are_bridges", "position"])
    col_index = pd.MultiIndex.from_product([unique_metrics, unique_strategies], names=["metric", "strategy"])
    
    # Step 4: Create the DataFrame
    df = pd.DataFrame(index=row_index, columns=col_index)
    
    # Step 5: Fill the DataFrame
    for are_val, pos_dict in merged_data.items():
        for pos, metric_strat_dict in pos_dict.items():
            for (m, s), val in metric_strat_dict.items():
                df.loc[(I_MAP, are_val, pos), (m, s)] = val
    
    return df
            

df_plotdicts = plot_all(df_map)
df_plotdicts

AssertionError: Series([], Name: probabilityForcingForHuman, dtype: float64)

<Figure size 2000x100 with 0 Axes>

In [ ]:
#IPython.display.display(IPython.display.HTML(df_plotdicts.to_html()))
df_plotdicts.to_csv(f'{DIRECTORY_DATA}/df_plotdicts_map{I_MAP}.csv')
print(df_plotdicts.columns)
print(df_plotdicts.index)

# Maps

In [ ]:
def show_maps(title, configurations, ncols): 
    nrows = max(1, (len(configurations) + ncols - 1) // ncols)
    fig, axes_matrix = plt.subplots(nrows, ncols, figsize=(16, 9), squeeze=False)
    #print(f'{title}: {nrows}x{ncols}')

    axes = list(itertools.chain.from_iterable(axes_matrix))
    axes_matrix: list[list[Axes]]
    assert len(axes) >= len(configurations)
    
    for ax in axes:
        ax.axis('off')
        
    for ax, configuration in zip(axes, configurations):
        filename_screenshot = configuration_to_filename_screenshot[configuration]
        image = plt.imread(filename_screenshot)
        ax.imshow(image)
        ax.title.set_text(f'Configuration:\n{configuration}')
    
    fig.suptitle(title, fontsize=16)    
    fig.tight_layout()
    fig.subplots_adjust(wspace=0.1, hspace=0.3)
    save_and_show(fig, title)
    

def show_all_maps():
    index_blocked, index_nonblocked = make_indexes_blocked_nonblocked(['configuration', 'are_bridges'])
    for title, index in {'Non-blocked': index_nonblocked, 'Blocked': index_blocked}.items():
        for are_bridges in True, False:
            show_maps(f'{title}: Map {I_MAP} ({"with" if are_bridges else "without"} bridges)', 
                      index[index.get_level_values('are_bridges') == are_bridges].get_level_values('configuration'), 
                      4)
            

show_all_maps()